# RZ Automedata - Media Upscale Server
### Real-ESRGAN on Google Colab GPU

### Setup:
1. **GPU Runtime**: `Runtime` > `Change runtime type` > `T4 GPU`
2. **Run All**: `Ctrl+F9`
3. Done! Watches Google Drive for jobs automatically.

**Supports both Video + Image upscaling!**

**No ngrok. No URL. Just run and go!**

---

In [ ]:
# @title 1. Install Dependencies

import subprocess, os, shutil

# --- GPU Check ---
try:
    r = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
                       capture_output=True, text=True, timeout=10)
    if r.returncode == 0:
        print(f'[OK] GPU: {r.stdout.strip()}')
    else:
        print('[WARNING] No GPU! Go to Runtime > Change runtime type > T4 GPU')
except FileNotFoundError:
    print('[WARNING] nvidia-smi not found! No GPU.')
except Exception as e:
    print(f'[WARNING] GPU check error: {e}')

# --- FFmpeg ---
!apt-get update -qq && apt-get install -y -qq ffmpeg > /dev/null 2>&1
print('[OK] FFmpeg')

# --- Real-ESRGAN ---
if not os.path.exists('/content/Real-ESRGAN'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git /content/Real-ESRGAN

%cd /content/Real-ESRGAN

!pip install -q basicsr facexlib gfpgan
!pip install -q -r requirements.txt
!python setup.py develop > /dev/null 2>&1

# --- Patch torchvision ---
def patch_basicsr_torchvision():
    import importlib
    spec = importlib.util.find_spec('basicsr')
    if spec is None or spec.origin is None:
        print('[WARNING] basicsr not found'); return
    basicsr_dir = os.path.dirname(spec.origin)
    deg_file = os.path.join(basicsr_dir, 'data', 'degradations.py')
    if not os.path.exists(deg_file): return
    with open(deg_file, 'r') as f: code = f.read()
    if 'functional_tensor' in code:
        code = code.replace(
            'from torchvision.transforms.functional_tensor import rgb_to_grayscale',
            'from torchvision.transforms.functional import rgb_to_grayscale')
        with open(deg_file, 'w') as f: f.write(code)
        print('[FIX] Patched basicsr (functional_tensor -> functional)')
    else:
        print('[OK] basicsr already patched')

patch_basicsr_torchvision()
import basicsr
print('[OK] basicsr imported')

# --- Patch torch.load for PyTorch 2.6+ compatibility ---
import torch
_orig_torch_load = torch.load
def _patched_load(*args, **kwargs):
    kwargs.setdefault("weights_only", False)
    return _orig_torch_load(*args, **kwargs)
torch.load = _patched_load
print("[OK] torch.load patched for PyTorch 2.6+")

# --- Download models ---
models_dir = '/content/Real-ESRGAN/weights'
os.makedirs(models_dir, exist_ok=True)

model_urls = {
    'RealESRGAN_x4plus.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth',
    'RealESRGAN_x4plus_anime_6B.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth',
    'realesr-animevideov3.pth': 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth',
    'GFPGANv1.3.pth': 'https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth'
}
for name, url in model_urls.items():
    path = os.path.join(models_dir, name)
    if not os.path.exists(path):
        !wget -q {url} -O {path}
        print(f'[OK] Downloaded {name}')
    else:
        print(f'[OK] {name} exists')

exp_dir = '/content/Real-ESRGAN/experiments/pretrained_models'
os.makedirs(exp_dir, exist_ok=True)
for name in model_urls:
    src = os.path.join(models_dir, name)
    dst = os.path.join(exp_dir, name)
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)

print('[OK] All models ready')

result = subprocess.run(
    ['python', '/content/Real-ESRGAN/inference_realesrgan.py', '--help'],
    capture_output=True, text=True, cwd='/content/Real-ESRGAN')
if result.returncode == 0:
    print('[OK] Real-ESRGAN verification PASSED!')
else:
    print('[FAILED] verification failed')
    print(result.stderr[-500:])

print('\nAll dependencies ready!')

In [ ]:
# @title 2. Mount Google Drive

from google.colab import drive
import os

drive.mount('/content/drive')

DRIVE_BASE   = '/content/drive/MyDrive/RZ_Upscaler'
DRIVE_INPUT  = os.path.join(DRIVE_BASE, 'Input')
DRIVE_OUTPUT = os.path.join(DRIVE_BASE, 'Output')
DRIVE_JOBS   = os.path.join(DRIVE_BASE, 'Jobs')
DRIVE_STATUS = os.path.join(DRIVE_BASE, 'Status')

for d in [DRIVE_INPUT, DRIVE_OUTPUT, DRIVE_JOBS, DRIVE_STATUS]:
    os.makedirs(d, exist_ok=True)

print('[OK] Google Drive mounted')
print(f'Input:  {DRIVE_INPUT}')
print(f'Output: {DRIVE_OUTPUT}')
print(f'Jobs:   {DRIVE_JOBS}')
print(f'Status: {DRIVE_STATUS}')

In [ ]:
# @title 3. Start Job Watcher
# @markdown Watches Google Drive Jobs/ folder for new jobs.
# @markdown **No ngrok needed!** All communication through Google Drive.
# @markdown **Supports Video + Image upscaling!**

import os, re, glob, shutil, subprocess, json, time, threading, traceback

DRIVE_BASE   = '/content/drive/MyDrive/RZ_Upscaler'
DRIVE_INPUT  = os.path.join(DRIVE_BASE, 'Input')
DRIVE_OUTPUT = os.path.join(DRIVE_BASE, 'Output')
DRIVE_JOBS   = os.path.join(DRIVE_BASE, 'Jobs')
DRIVE_STATUS = os.path.join(DRIVE_BASE, 'Status')
ESRGAN_DIR   = '/content/Real-ESRGAN'

# Image file extensions
IMAGE_EXTS = {'.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.tif', '.webp'}

# NVENC H.264 max resolution per dimension
NVENC_MAX_DIM = 4096

processed_jobs = set()

def is_image_file(filename):
    return os.path.splitext(filename)[1].lower() in IMAGE_EXTS

def get_gpu_info():
    try:
        r = subprocess.run(
            ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'],
            capture_output=True, text=True, timeout=5)
        if r.returncode == 0:
            parts = r.stdout.strip().split(',')
            return {'name': parts[0].strip(), 'memory': parts[1].strip() if len(parts) > 1 else ''}
    except: pass
    return {'name': 'Unknown', 'memory': ''}

GPU_INFO = get_gpu_info()
print(f'GPU: {GPU_INFO["name"]} ({GPU_INFO["memory"]})')


# =============================================
# STATUS + LOG
# =============================================

task_logs = {}

def log(task_id, msg):
    """Add log line and print."""
    if task_id not in task_logs:
        task_logs[task_id] = []
    task_logs[task_id].append(msg)
    print(f'  {msg}')

def update_status(task_id, status, progress=0, stage='', error=None):
    """Write status + full log to Drive Status/ folder."""
    data = {
        'task_id': task_id,
        'status': status,
        'progress': progress,
        'stage': stage,
        'error': error,
        'gpu': GPU_INFO['name'],
        'log': task_logs.get(task_id, []),
        'updated_at': time.time(),
    }
    status_path = os.path.join(DRIVE_STATUS, f'{task_id}.json')
    try:
        tmp = status_path + '.tmp'
        with open(tmp, 'w') as f:
            json.dump(data, f)
        os.replace(tmp, status_path)
    except:
        try:
            with open(status_path, 'w') as f:
                json.dump(data, f)
        except: pass


# =============================================
# FRAME COUNTER (for video)
# =============================================

_upscale_running = False

def _count_output_frames(frames_out, total_frames, task_id):
    global _upscale_running
    while _upscale_running:
        try:
            done = len(glob.glob(os.path.join(frames_out, '*.png')))
            if total_frames > 0 and done > 0:
                pct = min(12 + int((done / total_frames) * 63), 75)
                msg = f'[progress] {done}/{total_frames} ({pct}%)'
                # Replace previous progress line
                logs = task_logs.get(task_id, [])
                task_logs[task_id] = [l for l in logs if not l.startswith('[progress]')]
                task_logs[task_id].append(msg)
                update_status(task_id, 'processing', pct, f'Upscaling frame {done}/{total_frames}')
                print(f'  {msg}')
        except: pass
        time.sleep(3)


# =============================================
# ENCODER SELECTION HELPER
# =============================================

def select_encoder(out_w, out_h):
    """Select best encoder. Falls back to libx264 if resolution exceeds NVENC limit.
    Returns (encoder_args_list, encoder_label).
    """
    exceeds_limit = out_w > NVENC_MAX_DIM or out_h > NVENC_MAX_DIM

    hw_encoder = None
    encoder_label = 'libx264 (CPU)'

    if not exceeds_limit:
        try:
            enc_check = subprocess.run(['ffmpeg', '-hide_banner', '-encoders'],
                capture_output=True, text=True, timeout=5)
            for enc_name, enc_lbl in [('h264_nvenc','NVENC (NVIDIA)'),('h264_amf','AMF (AMD)'),('h264_qsv','QSV (Intel)')]:
                if enc_name in enc_check.stdout:
                    hw_encoder = enc_name
                    encoder_label = enc_lbl
                    break
        except: pass
    else:
        print(f'  [INFO] Output {out_w}x{out_h} exceeds NVENC limit ({NVENC_MAX_DIM}), using libx264')

    # Build encoder args — always use 50M bitrate for consistent quality
    if hw_encoder:
        if hw_encoder == 'h264_nvenc': preset_args = ['-preset', 'p1']
        elif hw_encoder == 'h264_amf': preset_args = ['-quality', 'speed']
        else: preset_args = ['-preset', 'veryfast']
        enc_args = ['-c:v', hw_encoder] + preset_args + [
            '-b:v', '50M', '-maxrate', '55M', '-bufsize', '100M', '-pix_fmt', 'yuv420p'
        ]
    else:
        enc_args = [
            '-c:v', 'libx264', '-b:v', '50M', '-maxrate', '55M',
            '-bufsize', '100M', '-preset', 'ultrafast', '-pix_fmt', 'yuv420p'
        ]

    return enc_args, encoder_label


# =============================================
# IMAGE PROCESSOR (direct upscale, no frames)
# =============================================

def process_image(job):
    """Process a single image — direct upscale, no frame extraction."""
    task_id = job['task_id']
    filename = job['filename']
    scale = job.get('scale', 4)
    model = job.get('model', 'realesr-animevideov3')
    face_enhance = job.get('face_enhance', False)
    output_format = job.get('output_format', 'png')
    target_fps = job.get('target_fps', 30)

    input_path = os.path.join(DRIVE_INPUT, filename)
    work_dir = f'/content/work_{task_id}'
    os.makedirs(work_dir, exist_ok=True)

    task_logs[task_id] = []
    log(task_id, f'Job: {filename} (IMAGE)')
    log(task_id, f'Model: {model} | Scale: {scale}x | GPU: {GPU_INFO["name"]}')

    try:
        # === WAIT FOR FILE ===
        update_status(task_id, 'waiting', 0, 'Waiting for image file...')
        t0 = time.time()
        while not os.path.exists(input_path):
            if time.time() - t0 > 600:
                raise Exception('File not found after 10 min')
            time.sleep(3)

        log(task_id, 'File found, verifying sync...')
        prev, stable = -1, 0
        while stable < 2:
            sz = os.path.getsize(input_path) if os.path.exists(input_path) else 0
            if sz > 0 and sz == prev: stable += 1
            else: stable = 0
            prev = sz
            time.sleep(2)

        mb = os.path.getsize(input_path) / (1024*1024)
        log(task_id, f'\u2713 Image ready: {mb:.1f} MB')
        update_status(task_id, 'processing', 5, 'Starting image upscale...')

        # === UPSCALE IMAGE DIRECTLY ===
        log(task_id, f'\u25b6 UPSCALING IMAGE ({scale}x with {model})')
        update_status(task_id, 'processing', 10, f'Upscaling image ({scale}x)...')

        out_dir = os.path.join(work_dir, 'output')
        os.makedirs(out_dir, exist_ok=True)

        cmd = [
            'python', os.path.join(ESRGAN_DIR, 'inference_realesrgan.py'),
            '-i', input_path, '-o', out_dir,
            '-n', model, '-s', str(scale),
            '--suffix', 'out', '--ext', output_format
        ]
        if 'x4plus' in model:
            cmd.extend(['--tile', '800'])
        if face_enhance:
            cmd.append('--face_enhance')

        t1 = time.time()
        update_status(task_id, 'processing', 20, 'Upscaling image on GPU...')

        proc = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            universal_newlines=True, bufsize=1, cwd=ESRGAN_DIR)

        output_lines = []
        for line in iter(proc.stdout.readline, ''):
            line = line.strip()
            if line:
                output_lines.append(line)
                task_logs[task_id].append(f'> {line}')
                print(f'  > {line}')
                update_status(task_id, 'processing', 50, 'Upscaling image on GPU...')
        proc.wait()
        upscale_time = time.time() - t1

        if proc.returncode != 0:
            err = '\n'.join(output_lines[-10:])
            raise Exception(f'ESRGAN failed: {err[:300]}')

        # Find output file
        out_files = glob.glob(os.path.join(out_dir, f'*_out.{output_format}'))
        if not out_files:
            # Try any file in output dir
            out_files = glob.glob(os.path.join(out_dir, '*.*'))
        if not out_files:
            raise Exception('No upscaled image found in output')

        upscaled_path = out_files[0]
        log(task_id, f'\u2713 Upscaled in {upscale_time:.1f}s')
        update_status(task_id, 'processing', 75, f'Upscaled in {upscale_time:.1f}s')

        # === COPY TO DRIVE OUTPUT ===
        out_name = f'{task_id}_UPSCALED.{output_format}'
        out_path = os.path.join(DRIVE_OUTPUT, out_name)

        log(task_id, f'Saving to Drive: {out_name}')
        update_status(task_id, 'processing', 85, 'Saving upscaled image to Drive...')
        shutil.copy2(upscaled_path, out_path)

        out_mb = os.path.getsize(out_path) / (1024*1024)
        total_time = time.time() - t0

        log(task_id, f'\u2713 Output: {out_name} ({out_mb:.1f} MB)')
        log(task_id, f'\u2705 DONE in {total_time:.1f}s!')
        update_status(task_id, 'completed', 100, f'Complete! ({out_mb:.1f} MB)')
        print(f'\n[DONE] {out_name} ({out_mb:.1f} MB) in {total_time:.1f}s')

    except Exception as e:
        log(task_id, f'\u274c ERROR: {str(e)[:150]}')
        update_status(task_id, 'failed', 0, f'Error: {str(e)[:150]}', str(e))
        print(f'[FAIL] {task_id}: {e}')
        traceback.print_exc()
    finally:
        if os.path.exists(work_dir):
            shutil.rmtree(work_dir, ignore_errors=True)
        if os.path.exists(input_path):
            try: os.remove(input_path)
            except: pass
        job_path = os.path.join(DRIVE_JOBS, f'{task_id}.json')
        try:
            if os.path.exists(job_path): os.remove(job_path)
        except: pass


# =============================================
# VIDEO PROCESSOR
# =============================================

def process_video(job):
    global _upscale_running
    task_id = job['task_id']
    filename = job['filename']
    scale = job.get('scale', 4)
    model = job.get('model', 'realesr-animevideov3')
    face_enhance = job.get('face_enhance', False)
    mute_audio = job.get('mute_audio', False)
    output_format = job.get('output_format', 'mp4')
    target_fps = job.get('target_fps', 30)

    input_path = os.path.join(DRIVE_INPUT, filename)
    work_dir = f'/content/work_{task_id}'
    frames_in = os.path.join(work_dir, 'frames_in')
    frames_out = os.path.join(work_dir, 'frames_out')
    os.makedirs(frames_in, exist_ok=True)
    os.makedirs(frames_out, exist_ok=True)

    task_logs[task_id] = []
    log(task_id, f'Job: {filename} (VIDEO)')
    log(task_id, f'Model: {model} | Scale: {scale}x | GPU: {GPU_INFO["name"]}')

    try:
        # === WAIT FOR FILE ===
        update_status(task_id, 'waiting', 0, 'Waiting for file...')
        t0 = time.time()
        while not os.path.exists(input_path):
            if time.time() - t0 > 600:
                raise Exception('File not found after 10 min')
            time.sleep(3)

        log(task_id, 'File found, verifying sync...')
        prev, stable = -1, 0
        while stable < 2:
            sz = os.path.getsize(input_path) if os.path.exists(input_path) else 0
            if sz > 0 and sz == prev: stable += 1
            else: stable = 0
            prev = sz
            time.sleep(2)

        mb = os.path.getsize(input_path) / (1024*1024)
        log(task_id, f'\u2713 File ready: {mb:.1f} MB')
        update_status(task_id, 'processing', 2, 'Analyzing video...')

        # === PHASE 1: EXTRACT FRAMES ===
        log(task_id, 'Analyzing video...')
        probe = subprocess.run(
            ['ffprobe', '-v', 'quiet', '-print_format', 'json',
             '-show_streams', '-show_format', input_path],
            capture_output=True, text=True)
        info = json.loads(probe.stdout)
        source_fps = 30.0
        has_audio = False
        width = 0
        height = 0
        for s in info.get('streams', []):
            if s.get('codec_type') == 'video':
                width = int(s.get('width', 0))
                height = int(s.get('height', 0))
                rf = s.get('r_frame_rate', '30/1')
                if '/' in str(rf):
                    n, d = str(rf).split('/')
                    try: source_fps = float(n)/float(d)
                    except: source_fps = 30.0
                else:
                    try: source_fps = float(rf)
                    except: source_fps = 30.0
            elif s.get('codec_type') == 'audio':
                has_audio = True

        # Default source FPS if not detected
        # target_fps=0 means 'Original' - keep source FPS
        effective_fps = target_fps if target_fps > 0 else int(round(source_fps))
        need_interpolation = target_fps > 0 and target_fps > source_fps + 1

        # Calculate output resolution
        out_w = width * scale
        out_h = height * scale

        log(task_id, f'\u2713 Video: {width}x{height} @ {source_fps:.0f} fps')
        log(task_id, f'\u2713 Output: {out_w}x{out_h} ({scale}x)')
        if has_audio:
            log(task_id, f'\u2713 Audio: detected{" (will be muted)" if mute_audio else ""}')

        log(task_id, 'Extracting frames...')
        update_status(task_id, 'processing', 5, 'Extracting frames...')
        t1 = time.time()
        r = subprocess.run([
            'ffmpeg', '-y', '-threads', '0', '-i', input_path,
            '-qscale:v', '1', '-qmin', '1', '-qmax', '1',
            os.path.join(frames_in, 'frame_%08d.png')
        ], capture_output=True, text=True)
        if r.returncode != 0:
            raise Exception('Frame extraction failed')
        total_frames = len(glob.glob(os.path.join(frames_in, '*.png')))
        if total_frames == 0:
            raise Exception('No frames extracted')
        extract_time = time.time() - t1
        log(task_id, f'\u2713 {total_frames} frames extracted in {extract_time:.1f}s')
        update_status(task_id, 'processing', 10, f'{total_frames} frames extracted')

        # === PHASE 2: UPSCALE ===
        log(task_id, f'\u25b6 UPSCALING: {width}x{height} -> {out_w}x{out_h} ({scale}x)')
        log(task_id, f'  Model: {model}')
        update_status(task_id, 'processing', 12, f'Upscaling {total_frames} frames...')
        _upscale_running = True

        cmd = [
            'python', os.path.join(ESRGAN_DIR, 'inference_realesrgan.py'),
            '-i', frames_in, '-o', frames_out,
            '-n', model, '-s', str(scale),
            '--suffix', 'out', '--ext', 'png'
        ]
        # x4plus uses much more VRAM than animevideov3, tile to avoid OOM
        if 'x4plus' in model:
            cmd.extend(['--tile', '800'])
        if face_enhance: cmd.append('--face_enhance')

        t2 = time.time()
        counter = threading.Thread(
            target=_count_output_frames,
            args=(frames_out, total_frames, task_id),
            daemon=True)
        counter.start()

        proc = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
            universal_newlines=True, bufsize=1, cwd=ESRGAN_DIR)

        output_lines = []
        for line in iter(proc.stdout.readline, ''):
            line = line.strip()
            if line:
                output_lines.append(line)
                task_logs[task_id].append(f'> {line}')
                print(f'  > {line}')
        proc.wait()
        _upscale_running = False
        upscale_time = time.time() - t2

        if proc.returncode != 0:
            err = '\n'.join(output_lines[-10:])
            raise Exception(f'ESRGAN failed: {err[:300]}')

        out_count = len(glob.glob(os.path.join(frames_out, '*.png')))
        if out_count == 0:
            raise Exception('No upscaled frames')

        log(task_id, f'\u2713 PHASE 2 COMPLETE: Upscaled {out_count} frames in {upscale_time:.1f}s')
        update_status(task_id, 'processing', 75, f'Upscaled {out_count} frames')

        # === PHASE 3: MERGE TO VIDEO ===
        log(task_id, '')
        log(task_id, f'\U0001f39e PHASE 3: MERGING TO VIDEO')
        log(task_id, f'  Merging {out_count} frames at {effective_fps} FPS{" (interpolated)" if need_interpolation else ""}')
        update_status(task_id, 'merging', 78, 'Merging frames to video...')

        out_name = f'{task_id}_UPSCALED.{output_format}'
        out_path = os.path.join(DRIVE_OUTPUT, out_name)

        # Frame pattern
        frame_pattern = os.path.join(frames_out, 'frame_%08d_out.png')
        test_frame = os.path.join(frames_out, 'frame_00000001_out.png')
        if not os.path.exists(test_frame):
            samples = sorted(glob.glob(os.path.join(frames_out, '*.png')))
            if samples:
                first = os.path.basename(samples[0])
                log(task_id, f'  Pattern fallback: {first}')
                import re as _re
                match = _re.search(r'(\d{6,})', first)
                if match:
                    num_str = match.group(1)
                    fmt_str = f'%0{len(num_str)}d'
                    frame_pattern = os.path.join(frames_out, first.replace(num_str, fmt_str))

        log(task_id, f'  Frame pattern: {os.path.basename(frame_pattern)}')

        # Build FFmpeg cmd with max threads for fast PNG decoding
        input_fps = str(int(round(source_fps)))
        ff = ['ffmpeg', '-y', '-threads', '0',
              '-framerate', input_fps, '-i', frame_pattern]
        if has_audio and not mute_audio:
            ff.extend(['-i', input_path, '-map', '0:v', '-map', '1:a?'])

        # FPS: framerate filter (fast) instead of minterpolate (slow)
        if need_interpolation:
            fps_ratio = effective_fps / max(source_fps, 1)
            if fps_ratio >= 1.8:
                log(task_id, f'  Interpolating {source_fps:.0f} -> {effective_fps} FPS')
                ff.extend(['-vf', f'framerate=fps={effective_fps}:interp_start=0:interp_end=255:scene=100'])
            else:
                log(task_id, f'  Adjusting {source_fps:.0f} -> {effective_fps} FPS')
                ff.extend(['-vf', f'fps={effective_fps}'])

        # Select encoder (auto-detects resolution limit)
        enc_args, encoder_label = select_encoder(out_w, out_h)
        log(task_id, f'  Encoder: {encoder_label}')
        if out_w > NVENC_MAX_DIM or out_h > NVENC_MAX_DIM:
            log(task_id, f'  [INFO] {out_w}x{out_h} exceeds NVENC limit, using CPU encoder')
        ff.extend(enc_args)

        if has_audio and not mute_audio:
            ff.extend(['-c:a', 'aac', '-b:a', '320k'])
        ff.extend(['-r', str(effective_fps)])
        ff.extend(['-movflags', '+faststart', out_path])
        log(task_id, f'  Encoding video...')
        update_status(task_id, 'merging', 80, f'Encoding with {encoder_label}...')

        t3 = time.time()
        ff_output_lines = []
        fp = subprocess.Popen(ff, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                              universal_newlines=True, bufsize=1)
        for line in iter(fp.stdout.readline, ''):
            ff_output_lines.append(line.strip())
            frame_match = re.search(r'frame=\s*(\d+)', line)
            if frame_match and total_frames > 0:
                cf = int(frame_match.group(1))
                pct = min(80 + int((cf/total_frames)*18), 98)
                stage_msg = f'Encoding frame {cf}/{total_frames}'
                update_status(task_id, 'merging', pct, stage_msg)
                task_logs[task_id] = [l for l in task_logs[task_id] if not l.startswith('  Encoding frame ')]
                task_logs[task_id].append(f'  Encoding frame {cf}/{total_frames}')
        fp.wait()
        encode_time = time.time() - t3

        if fp.returncode != 0:
            err_tail = '\n'.join(ff_output_lines[-5:])
            log(task_id, f'  FFmpeg ERROR: {err_tail}')
            raise Exception(f'FFmpeg failed (rc={fp.returncode}): {err_tail[:300]}')
        if not os.path.exists(out_path):
            raise Exception('Output file not created')

        out_mb = os.path.getsize(out_path) / (1024*1024)
        log(task_id, f'\u2713 PHASE 3 COMPLETE: Encoded in {encode_time:.1f}s')
        log(task_id, f'\u2713 Output: {out_name} ({out_mb:.1f} MB)')

        total_time = time.time() - t0
        log(task_id, '')
        log(task_id, f'\u2705 ALL DONE in {total_time:.1f}s!')
        log(task_id, f'  {width}x{height} -> {out_w}x{out_h} | {out_mb:.1f} MB')
        update_status(task_id, 'completed', 100, f'Complete! ({out_mb:.1f} MB)')
        print(f'\n[DONE] {out_name} ({out_mb:.1f} MB) in {total_time:.1f}s')

    except Exception as e:
        _upscale_running = False
        log(task_id, f'\u274c ERROR: {str(e)[:150]}')
        update_status(task_id, 'failed', 0, f'Error: {str(e)[:150]}', str(e))
        print(f'[FAIL] {task_id}: {e}')
        traceback.print_exc()
    finally:
        _upscale_running = False
        if os.path.exists(work_dir):
            shutil.rmtree(work_dir, ignore_errors=True)
        if os.path.exists(input_path):
            try: os.remove(input_path)
            except: pass
        job_path = os.path.join(DRIVE_JOBS, f'{task_id}.json')
        try:
            if os.path.exists(job_path): os.remove(job_path)
        except: pass


def _refresh_drive_cache(folder):
    """Force Google Drive FUSE to refresh directory listing.
    Write+delete a tiny temp file to invalidate the cache."""
    try:
        probe = os.path.join(folder, '.probe')
        with open(probe, 'w') as f: f.write('x')
        os.remove(probe)
    except: pass


def watch_jobs():
    print()
    print('=' * 55)
    print('  RZ UPSCALER - JOB WATCHER RUNNING')
    print(f'  GPU: {GPU_INFO["name"]} ({GPU_INFO["memory"]})')
    print(f'  Models: animevideov3, x4plus')
    print(f'  Supports: Video + Image')
    print(f'  Watching: {DRIVE_JOBS}')
    print('  Waiting for jobs from desktop app...')
    print('=' * 55)
    print()

    poll_count = 0
    while True:
        try:
            # Force-refresh Drive FUSE cache before scanning
            _refresh_drive_cache(DRIVE_JOBS)

            # Use os.listdir (more reliable than glob for FUSE)
            raw_files = os.listdir(DRIVE_JOBS)
            job_files = sorted([f for f in raw_files if f.endswith('.json') and not f.endswith('.tmp')])

            poll_count += 1
            if poll_count % 12 == 1:  # Log every ~60s
                print(f'[POLL #{poll_count}] Files in Jobs/: {raw_files if raw_files else "(empty)"}')

            for job_name in job_files:
                task_id = job_name.replace('.json', '')
                if task_id in processed_jobs: continue

                job_path = os.path.join(DRIVE_JOBS, job_name)
                try:
                    with open(job_path, 'r') as f:
                        job = json.load(f)
                except (json.JSONDecodeError, OSError):
                    continue

                filename = job.get('filename', '?')
                model_name = job.get('model', 'realesr-animevideov3')
                scale = job.get('scale', 4)
                file_type = 'IMAGE' if is_image_file(filename) else 'VIDEO'
                print(f'\n[NEW JOB] {task_id}: {filename} ({file_type}, model={model_name}, {scale}x)')
                processed_jobs.add(task_id)

                # Route to image or video processor
                if is_image_file(filename):
                    process_image(job)
                else:
                    process_video(job)

        except Exception as e:
            print(f'[ERROR] Job watcher: {e}')
            traceback.print_exc()

        time.sleep(5)

watch_jobs()
